In [ ]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
import keras.optimizers
from sklearn.metrics import classification_report
import keras.optimizers
from keras.applications import vgg16
import numpy as np
import random
import os
from tqdm import tqdm
import pickle
import cv2

In [ ]:
# Define necessary constants
TEST_DIR = '/content/Brain-Tumor-Classification-DataSet/Testing'
TRAIN_DIR = '/content/Brain-Tumor-Classification-DataSet/Training'
IMG_SIZE = 224
CATEGORIES = ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"]

In [ ]:
# Creating training dataset
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(TRAIN_DIR,category)
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
          img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
          training_data.append([new_array, class_num])

    random.shuffle(training_data)

create_training_data()
#np.save('train_data.npy', training_data)
print(len(training_data))

print("train")
print()
X_train = np.array([i[0] for i in training_data]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
Y_train = [i[1] for i in training_data]

pickle_out = open("X_train.pickle","wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("Y_train.pickle","wb")
pickle.dump(Y_train, pickle_out)
pickle_out.close()

100%|██████████| 827/827 [00:02<00:00, 326.38it/s]


2870
train



In [ ]:
# Creating testing dataset
testing_data = []

def create_testing_data():
    for category in CATEGORIES:
        path = os.path.join(TEST_DIR,category)
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):
          img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
          testing_data.append([new_array, class_num])

    random.shuffle(testing_data)

create_testing_data()
#np.save('testing_data.npy', testing_data)
print(len(testing_data))

print("testing")
print()
X_test= np.array([i[0] for i in testing_data]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
Y_test = [i[1] for i in testing_data]

pickle_out = open("X_test.pickle","wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

pickle_out = open("Y_test.pickle","wb")
pickle.dump(Y_test, pickle_out)
pickle_out.close()

100%|██████████| 74/74 [00:00<00:00, 105.79it/s]


394
testing



In [ ]:
# TRAIN MODEL

In [ ]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
import keras.optimizers
from sklearn.metrics import classification_report
import keras.optimizers
from keras.applications import vgg16
import numpy as np

In [ ]:
tensorboard = TensorBoard(log_dir='./VGG16logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
# We'll use VGG16 to quickly define a model
vgg16_model = vgg16.VGG16()
vgg16_model.summary()
type(vgg16_model)

553467096/553467096 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

keras.src.engine.functional.Functional

In [ ]:
# Dropping last layers
model = Sequential()
for layer in vgg16_model.layers[:-3]:
    model.add(layer)


for layer in model.layers:
    layer.trainable = True

# Adding last Dense Layer
model.add(Dense(4,activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer= "adam",
              metrics=['accuracy'],
              )

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=32, epochs=50)

Epoch 1/50
90/90 [==============================] - 72s 519ms/step - loss: 1.5520 - accuracy: 0.3794
Epoch 2/50
90/90 [==============================] - 34s 382ms/step - loss: 0.9196 - accuracy: 0.6035
Epoch 3/50
90/90 [==============================] - 34s 377ms/step - loss: 0.7711 - accuracy: 0.6666
Epoch 4/50
90/90 [==============================] - 34s 378ms/step - loss: 0.6678 - accuracy: 0.7105
Epoch 5/50
90/90 [==============================] - 34s 380ms/step - loss: 0.5867 - accuracy: 0.7624
Epoch 6/50
90/90 [==============================] - 34s 378ms/step - loss: 0.5396 - accuracy: 0.7812
Epoch 7/50
90/90 [==============================] - 34s 378ms/step - loss: 0.5119 - accuracy: 0.7955
Epoch 8/50
90/90 [==============================] - 34s 380ms/step - loss: 0.4712 - accuracy: 0.8098
Epoch 9/50
90/90 [==============================] - 34s 380ms/step - loss: 0.3949 - accuracy: 0.8394
Epoch 10/50
90/90 [==============================] - 34s 380ms/step - loss: 0.3795 - accura

In [ ]:
# Evaluate the model
scores = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

13/13 [==============================] - 4s 334ms/step - loss: 6.5314 - accuracy: 0.7360
Test loss: 6.53139066696167
Test accuracy: 0.7360405921936035


In [ ]:
model.save("model.h5") # replace model.h5 in "Runner"

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_bool))

7/7 [==============================] - 12s 187ms/step
              precision    recall  f1-score   support

           0       0.91      0.21      0.34       100
           1       0.71      0.97      0.82       115
           2       0.68      1.00      0.81       105
           3       0.88      0.70      0.78        74

    accuracy                           0.74       394
   macro avg       0.80      0.72      0.69       394
weighted avg       0.79      0.74      0.69       394



In [ ]:
y_pred_bool

array([1, 1, 1, 3, 1, 3, 0, 2, 3, 3, 0, 2, 2, 2, 2, 3, 2, 2, 2, 1, 2, 1,
       2, 0, 1, 1, 1, 1, 2, 1, 3, 2, 1, 2, 3, 2, 1, 2, 1, 0, 1, 1, 3, 2,
       3, 1, 1, 2, 2, 1, 3, 3, 3, 2, 3, 1, 1, 2, 3, 2, 2, 2, 0, 2, 1, 1,
       0, 1, 3, 1, 1, 3, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2,
       2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 3, 2, 2, 1, 2, 1, 1, 1, 3, 2, 3, 1,
       3, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 0, 2, 1, 0, 2, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 3, 1, 3, 1, 3, 1, 2, 1, 1, 1, 3, 3, 1, 3, 2, 1, 2, 2, 2, 3,
       2, 2, 0, 3, 2, 1, 1, 2, 0, 0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 3, 2, 2,
       1, 2, 1, 2, 1, 1, 0, 2, 3, 2, 2, 0, 1, 2, 2, 0, 2, 2, 2, 2, 1, 2,
       1, 1, 2, 2, 2, 1, 3, 3, 2, 2, 1, 2, 1, 2, 1, 3, 2, 2, 2, 1, 1, 1,
       2, 1, 1, 3, 2, 3, 3, 2, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 2, 2, 1, 3,
       2, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 2, 3, 1, 2, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 1, 3, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1,